<a href="https://colab.research.google.com/github/Spili11/DS_Praktikum/blob/main/assignment_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical
from google.colab import drive

from tensorflow.keras.preprocessing import image_dataset_from_directory


In [2]:
drive.mount('/content/drive')

NameError: ignored

In [26]:
import os
import shutil
import random

Data splitting

In [22]:
mypath = "/content/drive/My Drive/DS_Praktikum/assignment05_data/"
a = ['train', 'validation']
b = ['batman', 'superman', 'spiderman']
for x, y in [(x,y) for x in a for y in b]:
  if not os.path.isdir(mypath + x + "/"+ y):
    os.makedirs(mypath + x + "/" + y)

In [132]:
validation_ratio = 0.2

batman_raw = os.listdir("/content/drive/My Drive/DS_Praktikum/assignment05_data/batman")
batman_validation = random.sample(batman_raw, k = int((len(batman_raw) * validation_ratio) // 1))
batman_train = list(set(batman_raw) - set(batman_validation))

superman_raw = os.listdir("/content/drive/My Drive/DS_Praktikum/assignment05_data/superman")
superman_validation = random.sample(superman_raw, k = int((len(superman_raw) * validation_ratio) // 1))
superman_train = list(set(superman_raw) - set(superman_validation))

spiderman_raw = os.listdir("/content/drive/My Drive/DS_Praktikum/assignment05_data/spiderman")
spiderman_validation = random.sample(spiderman_raw, k = int((len(spiderman_raw) * validation_ratio) // 1))
spiderman_train = list(set(spiderman_raw) - set(spiderman_validation))


In [133]:
print(len(batman_raw), len(batman_validation), len(batman_train))
print(len(spiderman_raw), len(spiderman_validation), len(spiderman_train))
print(len(superman_raw), len(superman_validation), len(superman_train))

292 58 234
290 58 232
292 58 234


In [137]:
for x in batman_train:
  original = "/content/drive/My Drive/DS_Praktikum/assignment05_data/batman/" + x
  target = "/content/drive/My Drive/DS_Praktikum/assignment05_data/train/batman/" + x
  
  shutil.copyfile(original, target)

In [138]:
for x in batman_validation:
  original = "/content/drive/My Drive/DS_Praktikum/assignment05_data/batman/" + x
  target = "/content/drive/My Drive/DS_Praktikum/assignment05_data/validation/batman/" + x
  
  shutil.copyfile(original, target)

In [139]:
for x in superman_train:
  original = "/content/drive/My Drive/DS_Praktikum/assignment05_data/superman/" + x
  target = "/content/drive/My Drive/DS_Praktikum/assignment05_data/train/superman/" + x
  
  shutil.copyfile(original, target)

In [140]:
for x in superman_validation:
  original = "/content/drive/My Drive/DS_Praktikum/assignment05_data/superman/" + x
  target = "/content/drive/My Drive/DS_Praktikum/assignment05_data/validation/superman/" + x
  
  shutil.copyfile(original, target)

In [141]:
for x in spiderman_train:
  original = "/content/drive/My Drive/DS_Praktikum/assignment05_data/spiderman/" + x
  target = "/content/drive/My Drive/DS_Praktikum/assignment05_data/train/spiderman/" + x
  
  shutil.copyfile(original, target)

In [142]:
for x in spiderman_validation:
  original = "/content/drive/My Drive/DS_Praktikum/assignment05_data/spiderman/" + x
  target = "/content/drive/My Drive/DS_Praktikum/assignment05_data/validation/spiderman/" + x
  
  shutil.copyfile(original, target)

In [3]:
train_dir = "/content/drive/My Drive/DS_Praktikum/assignment05_data/train/"
validation_dir = "/content/drive/My Drive/DS_Praktikum/assignment05_data/validation/"

BATCH_SIZE = 32
EPOCHS = 5
IMG_SIZE = (160, 160)
IMG_SHAPE = IMG_SIZE + (3,)

MobileNetV2


In [4]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
base_model.summary()

9412608/9406464 [==============================] - 0s 0us/step
Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 161, 161, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
________________

Transfer learning

In [5]:
for layer in base_model.layers:
    layer.trainable = False
    
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(units=3, activation="softmax")(x)

model = tf.keras.Model(inputs=base_model.input, outputs=outputs)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [7]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    class_mode='categorical',
    batch_size=32
)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir, 
    target_size=IMG_SIZE,
    class_mode='categorical',
    batch_size=32
)

Found 700 images belonging to 3 classes.
Found 174 images belonging to 3 classes.


Training

In [9]:
history = model.fit(train_generator,
                    epochs=EPOCHS,
                    validation_data=validation_generator)

Epoch 1/5


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


22/22 [==============================] - 178s 8s/step - loss: 0.8193 - accuracy: 0.6443 - val_loss: 0.6668 - val_accuracy: 0.7241
Epoch 2/5
22/22 [==============================] - 34s 2s/step - loss: 0.6180 - accuracy: 0.7500 - val_loss: 0.5453 - val_accuracy: 0.8046
Epoch 3/5
22/22 [==============================] - 34s 2s/step - loss: 0.5056 - accuracy: 0.8071 - val_loss: 0.4844 - val_accuracy: 0.8218
Epoch 4/5
22/22 [==============================] - 34s 2s/step - loss: 0.4539 - accuracy: 0.8243 - val_loss: 0.4605 - val_accuracy: 0.8391
Epoch 5/5
22/22 [==============================] - 34s 2s/step - loss: 0.4116 - accuracy: 0.8457 - val_loss: 0.4586 - val_accuracy: 0.8448
